In [1]:
from scipy.io import loadmat
a = loadmat('texas.mat')


In [9]:
a['A'][0,:]

array([0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1], dtype=uint8)

In [4]:
a.keys()

dict_keys(['__header__', '__version__', '__globals__', 'A', 'label', 'F'])

In [13]:
import numpy as np
np.max(a['F'][0,:])

1

In [7]:
a['label']

array([[1],
       [3],
       [5],
       [1],
       [2],
       [5],
       [5],
       [5],
       [2],
       [1],
       [2],
       [3],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [3],
       [5],
       [5],
       [5],
       [3],
       [5],
       [3],
       [5],
       [2],
       [1],
       [5],
       [5],
       [5],
       [2],
       [3],
       [2],
       [5],
       [1],
       [5],
       [2],
       [5],
       [5],
       [5],
       [5],
       [5],
       [2],
       [1],
       [5],
       [5],
       [5],
       [5],
       [2],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [1],
       [5],
       [5],
       [5],
       [5],
       [5],
       [1],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [1],
       [2],
       [2],
       [3],
       [2],
       [1],
       [3],
       [2],
       [1],
       [5],
       [2],
       [1],
       [1],
    